In [1]:
import math
import os

from fpgrowth_py import fpgrowth
import pandas as pd
import glob
import json
import spacy
from flashtext import KeywordProcessor
import ahocorasick
import nltk
from concurrent.futures import ThreadPoolExecutor
import plotly.graph_objects as go

pathprefix = r"D:\Code Working Area\Jupyter\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\ExcelData"
jsfile = pathprefix+"\\Transactions.json"
stop_words=pathprefix+"\\stop_words\\stop_words_stakeholder.txt"
unknow_long_phrase=pathprefix+"\\stop_words\\unknow_long.txt"

project_sor = pathprefix+"\\Source\\project.csv"
risk_sor = pathprefix+"\\Source\\risk.xlsx"
stake_sor = pathprefix+"\\Source\\stakeholder.csv"

project_key = pathprefix+"\\project_keyword\\Project_keyword.xlsx"
risk_key_path = pathprefix+"\\risk_keyword\\Risk_keyword.xlsx"
stake_key = pathprefix+"\\stakeholder_keyword\\third_layer_iteration_one_stakeholder.xlsx"

In [2]:
# !pip install regex
# !pip install spacy
# !pip install fpgrowth_py
# !pip install pandas
# !pip install seaborn
# !pip install openpyxl
# !python -m spacy install en_core_web_sm

In [3]:
prj_sor = pd.read_csv(project_sor, sep = ",")
risk_sor = pd.read_excel(risk_sor)
stk_sor = pd.read_csv(stake_sor, sep = ",")

prj_key = pd.read_excel(project_key, index_col=None)
risk_key = pd.read_excel(risk_key_path, index_col=None)
stk_key = pd.read_excel(stake_key, index_col=None)
stk_key.dropna(inplace=True)

C:\Users\jqi22\AppData\Local\Temp\ipykernel_17688\3799099859.py:1: DtypeWarning: Columns (3,7,8,12,13,18,27,28,29,30,31,38,39,40,41,42,43,44,45,50,51,53,54,59,63,64,65,67) have mixed types. Specify dtype option on import or set low_memory=False.
  prj_sor = pd.read_csv(project_sor, sep = ",")


In [4]:
# get the target text from original dataset to match
project = pd.DataFrame(prj_sor["Article Title"])
risk = pd.DataFrame(risk_sor["Abstract"])
stake = pd.DataFrame(stk_sor["Abstract"])

risk.dropna(inplace=True, how="any")

In [5]:
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

with open(stop_words, "r") as file:
    in_list=[val.strip() for val in file.readlines()]

def reload_dashes(droped: int)->set:
    dashes = pd.read_excel(pathprefix+"\\stop_words\\adjustment.xlsx")
    dashes = dashes[dashes.frequency > droped].Words.to_list()
    return set(dashes)

stopwords |= reload_dashes(2)
stopwords |= set([str(num) for num in range(1,100)])
stop_words={*stop_words, *in_list}

# manually add the words to the stopwords
possiblew = {"connections", "efficacy", "life", "This"}
stopwords |= possiblew

In [6]:
stk_key.Abstract = stk_key.Abstract.str.lower()

# Do I really need below two steps?
risk_key.Abstract=risk_key.Abstract.astype(str)
stk_key.Abstract=stk_key.Abstract.astype(str)

### First section of extraction keyword finished
#### Usually suggested to comment out if already with test dataset

In [7]:
# test_file=pd.DataFrame([])
# test_file=test_file.replace(regex=["{", "}", "'"], value="")
def Warp_fpgrowth(minSup:float, confident:float, dataset_name: str):
    ...

In [8]:
# Originally be devised for initialize the keyword for later integration
class keyword():
    prj, stk, risk="project", "stakeholder", "risk"
    prjw: KeywordProcessor=None
    stkw: KeywordProcessor=None
    riskw: KeywordProcessor=None

    def re_estabilishment(self):
        if not self.prjw and not self.stkw and not self.riskw:
            ...


In [9]:
# test running process
prjkeyword, stkeyw, riskeyw=KeywordProcessor(), KeywordProcessor(), KeywordProcessor()
prjkeyword.add_keywords_from_list(prj_key.Article.to_list())
stkeyw.add_keywords_from_list(stk_key.Abstract.to_list())
riskeyw.add_keywords_from_list(risk_key.Abstract.to_list())

In [10]:
def single_plot(left: KeywordProcessor, val: list):
    existence: int=0
    for keyword in val:
        if left.extract_keywords(keyword): existence+=1
    return existence

In [ ]:
def row_retrieve(name: str, left: KeywordProcessor, comparison: pd.DataFrame):
    extradata: list=[]
    for val in comparison.iterrows():
        ori, dest = val[1][0:2]
        ori,dest=ori.split(", "), dest.split(", ")
        ori,dest=single_plot(left, ori), single_plot(left, dest)
        extradata.append([ori, dest])
    comparison[[name+" on start", name+" on destination"]] = pd.DataFrame(extradata)
    return comparison

In [12]:
def test_retrieve(test_file: pd.DataFrame):
    temp=row_retrieve("prj", prjkeyword, test_file)
    temp=row_retrieve("stk", stkeyw, temp)
    return row_retrieve("risk", riskeyw, temp)

In [13]:
def general_info(dataset:pd.DataFrame):
    series=dataset.loc[:,dataset.columns.to_list()[3:]]
    remain=series[series.sum(axis=1)>1]
    len3= remain.apply(lambda row: 1 if row[0]+row[1]>0 and row[2]+row[3]>0 and row[4]+row[5]>0 else 0, axis=1).sum()
    len2=remain.apply(lambda row: 1 if sum([1 if row[i]+row[j]>0 else 0 for i in [0,2,4] for j in [1,3,5] if i==j-1])==2 else 0, axis=1).sum()
    remain_val=(len2+len3)/len(series)
    return [remain, remain_val, len2, len3]

In [14]:
def general_info_judge(dataset:pd.Series):
    series=dataset.to_list()[3:]
    len3= True if series[0]+series[1]>0 and series[2]+series[3]>0 and series[4]+series[5]>0 else False
    len2=True if sum([1 if series[i]+series[j]>0 else 0 for i in [0,2,4] for j in [1,3,5] if i==j-1])==2 else False
    return True if len2 or len3 else False

In [15]:
# start, des=[],[]
# for val in range(0, 6):
# if info[val]&(val%2==0): start.append(val)
# elif info[val]&val%2: des.append(val)
def eigen_counts(remain: pd.Series):
    p2sr, sp2r, pr2s, p2s, p2r, s2r, r2s=0,0,0,0,0,0,0
    for _, val in remain.iterrows():
        info=val.to_list()
        if info[0]&info[3]&info[5]: p2sr+=1
        elif info[2]&info[0]&info[5]: sp2r+=1
        elif info[0]&info[4]&info[3]: pr2s+=1
        elif info[0]&info[3]&(not info[1]+info[2]+info[4]+info[5]): p2s+=1
        elif info[0]&info[5]&(not info[1]+info[2]+info[4]+info[3]): p2r+=1
        elif info[2]&info[5]&(not info[1]+info[0]+info[4]+info[3]): s2r+=1
        elif info[4]&info[3]&(not info[1]+info[2]+info[0]+info[5]): r2s+=1
    return [p2sr, sp2r, pr2s, p2s, p2r, s2r, r2s]

In [16]:
# remain, remain_val, len2, len3=general_info(temp)
# pd.DataFrame([len(temp), remain_val, len2, len3, *eigen_counts(remain)], columns=["values"], index=["file size", "validate rate", "length larger 2", "length larger than 3", "p2sr", "sp2r", "pr2s", "p2s", "p2r", "s2r", "r2s"])

In [ ]:
file_name=os.listdir(pathprefix+"\\FP_growth_result")
initial=pd.DataFrame([], index=["file size", "validate rate", "length larger 2", "length larger than 3", "p2sr", "sp2r", "pr2s", "p2s", "p2r", "s2r", "r2s"])
for files in file_name:
    doc=pd.read_csv(pathprefix+f"\\FP_growth_result\\{files}").replace(regex=["{", "}", "'"], value="")
    # print(files)
    if doc.shape[1]!=3: continue
    counting=test_retrieve(doc)
    remain, remain_val, len2, len3=general_info(counting)
    initial[files]=[len(doc),remain_val, len2, len3, *eigen_counts(remain)]

In [28]:
initial

,prj_0.0001049979_0.18.csv,prj_0.0001049979_0.22.csv,prj_0.0001049979_0.26.csv,prj_0.0001060305_0.1.csv,prj_0.0001060305_0.22.csv,prj_0.0001060305_0.26.csv,prj_0.0001312474_0.18.csv,prj_0.0001325381_0.14.csv,prj_0.0001574969_0.26.csv,prj_0.0001590457_0.18.csv,...,stk_0.0003306468_0.26.csv,stk_0.0003306468_0.3.csv,stk_0.0003665689_0.18.csv,stk_0.0003665689_0.22.csv,stk_0.0003665689_0.26.csv,stk_0.0003719777_0.14.csv,stk_0.0003719777_0.22.csv,stk_0.0004072988_0.14.csv,stk_0.0004072988_0.3.csv,stk_0.0004133085_0.14.csv
file size,89.00000,82.00000,76.000000,40.000,27.00000,25.00,21.000000,35.000000,10.0,21.000000,...,39.000000,35.000000,39.000000,34.000000,32.00000,33.000000,26.000000,33.000000,22.000000,33.000000
validate rate,0.47191,0.47561,0.486842,0.425,0.37037,0.36,0.285714,0.428571,0.3,0.285714,...,0.230769,0.257143,0.128205,0.147059,0.15625,0.151515,0.192308,0.151515,0.227273,0.151515
length larger 2,42.00000,39.00000,37.000000,17.000,10.00000,9.00,6.000000,15.000000,3.0,6.000000,...,9.000000,9.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000
length larger than 3,0.00000,0.00000,0.000000,0.000,0.00000,0.00,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
p2sr,0.00000,0.00000,0.000000,0.000,0.00000,0.00,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
sp2r,0.00000,0.00000,0.000000,0.000,0.00000,0.00,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
pr2s,0.00000,0.00000,0.000000,0.000,0.00000,0.00,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
p2s,12.00000,11.00000,11.000000,7.000,4.00000,4.00,3.000000,6.000000,2.0,3.000000,...,3.000000,3.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000
p2r,0.00000,0.00000,0.000000,0.000,0.00000,0.00,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
s2r,5.00000,5.00000,4.000000,3.000,2.00000,2.00,1.000000,3.000000,0.0,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000


In [82]:
# files=file_name[1]
# doc=pd.read_csv(pathprefix+"\\FP_growth_result\\risk_9.92605e-05_0.26.csv").replace(regex=["{", "}", "'"], value="")
# counting=test_retrieve(doc)
# remain, remain_val, len2, len3=general_info(counting)
# initial[files]=[len(remain),remain_val, len2, len3, *eigen_counts(remain)]

1.453

In [29]:
partition=initial.loc[["validate rate"], :]
value, columns=partition.values, partition.columns.to_list()

In [30]:
new_vis_set=[columns[num].replace(".csv", "").split("_")+[value[0][num]] for num in range(0, len(columns))]

In [31]:
new_vis_set=pd.DataFrame(new_vis_set, columns=["name", "fre", "conf", "validate rate"])
new_vis_set[["fre", "conf"]]=new_vis_set.loc[:, ["fre", "conf"]].astype(float)

In [32]:
new_vis_set

,name,fre,conf,validate rate
0,prj,0.000105,0.18,0.471910
1,prj,0.000105,0.22,0.475610
2,prj,0.000105,0.26,0.486842
3,prj,0.000106,0.10,0.425000
4,prj,0.000106,0.22,0.370370
...,...,...,...,...
84,stk,0.000372,0.14,0.151515
85,stk,0.000372,0.22,0.192308
86,stk,0.000407,0.14,0.151515
87,stk,0.000407,0.30,0.227273


In [132]:
# # Read data from a csv
# z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')
#
# fig = go.Figure(data=[go.Surface(z=z_data.values)])
# fig.update_traces(contours_z=dict(show=True, usecolormap=True,
#                                   highlightcolor="limegreen", project_z=True))
# fig.update_layout(title='Mt Bruno Elevation', autosize=False,
#                   scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
#                   width=500, height=500,
#                   margin=dict(l=65, r=50, b=65, t=90)
# )

# fig.show()

In [33]:
# vals=attempt1.values
fig=go.Figure(data=[go.Surface(
    y=new_vis_set.conf.values,  # (5, 40, 5)
    x=new_vis_set.fre.values,  # (10, 30, 5)
    z=new_vis_set["validate rate"].values, name="Test validation Score")
])

In [ ]:
fig.update_layout(title='Validation Score', autosize=False,
                  scene_camera_eye=dict(x=1.7, y=1.8, z=-0.5),
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene = dict(
                      yaxis_title='Condition Probability',
                      xaxis_title='Frequency',
                      zaxis_title='Scores')
                  )
# , x="n_estimator", y="max_depth", z="Scores", color="species", title="validation plot"
# fig.show()

In [35]:
import plotly.express as px
fig=px.scatter_3d(new_vis_set, x="fre", y="conf", z="validate rate", color="name", opacity=0.7)
fig.show()

In [ ]:
def keep_validate(df: pd.Series):
    ...

In [31]:
def data_summary(df: pd.DataFrame):
    sp2r=(df["stk on start"].sum()+df["prj on start"].sum())/df["risk on start"].sum()
    pr2s=(df["risk on start"].sum()+df["prj on start"].sum())/df["stk on start"].sum()
    p2rs=df["prj on start"].sum()/(df["risk on start"].sum()+df["stk on start"].sum())
    return pd.DataFrame([sp2r, pr2s, p2rs], index=["sp2r", "pr2s", "p2rs"], columns=["convertion rate"])

In [ ]:
def split_hor(df: pd.DataFrame, number: int=10) -> list:
    if df.empty(): raise Exception("dataframe is empty.")
    interval=math.floor(df.shape[1]/number)
    return [df.loc[val*interval:val*(interval+1), ('p1', "p2", 'probability')] for val in range(0, number)]

### multithreading support

In [ ]:
def falshretrieve(left: KeywordProcessor, comparison: list):
    with ThreadPoolExecutor() as executor:
        result=executor.map(row_retrieve, left, comparison)


In [83]:
def initial_keyw():
    riskw, projectw, stkw=KeywordProcessor(),KeywordProcessor(),KeywordProcessor()
    keywords=keyword()
    keywords.riskw, keywords.prjw=riskw.add_keywords_from_list(riskw), projectw.add_keywords_from_list(prjkeyw)
    keywords.stkw= stkw.add_keywords_from_list(stkkeyw)
    return keywords
    # return [riskw.add_keywords_from_list(riskw),
    #         projectw.add_keywords_from_list(prjkeyw),
    #         stkw.add_keywords_from_list(stkkeyw)]

In [ ]:
def outer(comparison: pd.DataFrame):
    """
    multithreading must involve in the last
    :param comparison:
    :return:
    """
    keywords=initial_keyw()
    flash=split_hor(comparison, 8)
    for processor in keywords:
        df=falshretrieve(processor, flash)
    return df

In [71]:
def check_in():
    warpfp=Warp_fpgrowth(0.01, 0.2, "res")
    callAC=acmachine()

    ...

In [75]:
if __name__=="__main__":
    keys=initial_keyw()


0         [TIME, grouting risk, PURPOSE, Dynamic risk, M...
1         [TAR, ON, Safety, And risk, uses risk, Operati...
2         [ON, Safety, The risk, Resource, To risk, Unde...
3         [AIM, Aggregate risk, Paramount, Planning, The...
4         [Weibull, ON, Networks, COST, And risk, Differ...
                                ...                        
125092    [TIMES, Elsevier Ltd., Characteristics, Differ...
125093    [ON, National, Utilities, Elsevier Inc., Post,...
125094    [ON, complete risk, The risk, Economics, With ...
125095    [Sustainability, ON, epidemic risk, The risk, ...
125096    [investor risk, ON, Credit risk, EU, Resulting...
Name: Abstract, Length: 120420, dtype: object